In [1]:
import torch
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import MultivariateEvaluator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error
from typing import List

sys.path.append("..")
from timefusion.utils import metrics

In [2]:
# Import dataset
train_data = pd.read_csv("../datasets/electricity/train.csv", index_col="date")
val_data = pd.read_csv("../datasets/electricity/val.csv", index_col="date")
test_data = pd.read_csv("../datasets/electricity/test.csv", index_col="date")

# Normalize the signal power of each column
stds = train_data.std()
train_data /= stds
val_data /= stds
test_data /= stds

In [3]:
context_length = 48
prediction_length = 24

# Convert data into a glounts ListDataset
def get_dataset(df: pd.DataFrame, freq: str = "h", indices: List[int] = [-1]) -> ListDataset:
    return ListDataset(
        [
            {
                "start": df.index[0],
                "target": df.values[:i].T,
            }
            for i in indices
        ],
        freq=freq,
        one_dim_target=False
    )

train_dataset = get_dataset(train_data)
val_dataset = get_dataset(val_data)
test_dataset = get_dataset(test_data, indices=list(range(test_data.shape[0], test_data.shape[0] - 14*prediction_length, -prediction_length)))


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
estimator = TransformerTempFlowEstimator(
    d_model=32,
    num_heads=4,
    input_size=1280,
    target_dim=train_data.shape[1],
    prediction_length=prediction_length,
    context_length=context_length,
    hidden_size = 64,
    n_blocks=5,
    dropout_rate = 0.1,
    num_encoder_layers=5,
    num_decoder_layers=1,
    dim_feedforward_scale = 4,
    n_hidden=2,
    conditioning_length=128,
    dequantize=False,
    flow_type='RealNVP',
    freq="h",
    trainer=Trainer(
        device=device,
        epochs=2,
        learning_rate=1e-3,
        num_batches_per_epoch=100,
        batch_size=64,
    ),
)

In [6]:
# Train model
predictor = estimator.train(train_dataset,num_workers=2)

  0%|          | 0/99 [00:00<?, ?it/s]

/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated 

  0%|          | 0/99 [00:00<?, ?it/s]

/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:324: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(timestamp_input, freq=freq)
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version.
  if isinstance(timestamp.freq, Tick):
/rds/general/user/ejh19/home/Final-Year-Project/transformer_maf/.venv/lib/python3.8/site-packages/gluonts/dataset/common.py:329: FutureWarning: Timestamp.freq is deprecated and will be removed in a future ve

hello


In [11]:

start = time.time()
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_dataset,
    predictor=predictor,
    num_samples=128
)

samples = list(forecast_it)
realisations = list(ts_it)

print(time.time()-start)


0.3992013931274414


In [8]:
samples = torch.tensor([sample.samples for sample in samples]).permute(0,1,3,2)
realisations = torch.tensor([real.values[-prediction_length:] for real in realisations]).permute(0,2,1)

/var/tmp/pbs.7698056.pbs/ipykernel_2607307/1595039056.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  samples = torch.tensor([sample.samples for sample in samples]).permute(0,1,3,2)
